<a href="https://colab.research.google.com/github/mr-cri-spy/Movie_seat_enq_bot/blob/main/Movie_bot_updated_with_seat_availability_Small_UI_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install needed packages
!pip install spacy gradio
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 45.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
import spacy
import gradio as gr
from bs4 import BeautifulSoup
import random

nlp = spacy.load("en_core_web_sm")

# NLP Entity Extractor
def extract_movie_city_spacy(text):
    doc = nlp(text)
    movie = city = None
    for ent in doc.ents:
        if ent.label_ == "GPE":
            city = city or ent.text
        elif ent.label_ in ["ORG", "WORK_OF_ART"]:
            movie = movie or ent.text
    return movie, city


In [8]:
def simulate_multiple_theatres(movie, city):
    theatre_list = ["Kingdome", "Inox Forum", "PVR Mall", "DRC Cinemas"]
    html_blocks = []
    for theatre in theatre_list:
        total = random.randint(80, 200)
        booked = random.randint(10, total - 20)
        available = total - booked
        seat_html = f"""
        <div class="theatre">
            <h3>{movie or 'Some Movie'} - {theatre}, {city or 'City'}</h3>
            <div class="seats" data-total="{total}" data-booked="{booked}" data-available="{available}"></div>
        </div>
        """
        html_blocks.append(seat_html)
    return "\n".join(html_blocks)

def parse_all_theatres(html):
    soup = BeautifulSoup(html, "html.parser")
    theatres = soup.find_all("div", class_="theatre")
    parsed = []
    for theatre in theatres:
        name = theatre.find("h3").text.strip()
        seats = theatre.find("div", class_="seats")
        total = int(seats['data-total'])
        booked = int(seats['data-booked'])
        available = int(seats['data-available'])
        parsed.append((name, total, booked, available))
    return parsed


In [9]:
def crisbee_moviebot_response(user_input):
    movie, city = extract_movie_city_spacy(user_input)
    if not movie and not city:
        return " Sorry, I couldn't detect a movie or city in your question."

    html = simulate_multiple_theatres(movie, city)
    results = parse_all_theatres(html)

    if not results:
        return " No shows found right now."

    response = f" **Results for '{movie}' in {city}**:\n\n"
    for name, total, booked, available in results:
        response += f" {name}\n Total: {total} |  Booked: {booked} |  Available: {available}\n\n"
    return response


In [10]:
gr.Interface(
    fn=crisbee_moviebot_response,
    inputs=gr.Textbox(label="Ask about movie seat availability (e.g., 'Any Kalki shows in Mysore?')"),
    outputs=gr.Textbox(label="Crisbee MovieBot Response"),
    title=" Crisbee Movie Seat Assistant",
    description="Ask in natural language about movie availability in any city!"
).launch()

#Are there any Kalki shows in Mysore?


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://18dffdaf08d5b580b2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
